# P-POIS Optimization test
In this notebook we will build an optimization procedure for an agent that has the goal of keeping the Como Lake at a constant level. The inflow values that will be used start in 1946 and ends in 2011. More recent values can be found at this [link](https://adda.laghi.net/homepage.aspx?tab=3&subtab=2&idlago=1).

In [2]:
# Import stuff
import numpy as np
import scipy.stats
import os

## Como Lake RL Environment
We start by building the environment of the lake, which at each timestep will be represented by its water level (which will be the state $s_t$ and by the inflow $i_t$ of that day. In particular, the agent will choose its action $a_t$ knowing the current level of the lake but not the inflow that will be measured during that day. The reward $r_t$ will simply be equal the difference between the current level and the zero-level $s_0=197.37m$ of the Comolake.

The update of the state is given by the following equation:

$$ s_{t+1} = s_t + \frac{(~\text{inflow}_t - \text{outflow}_t~)~n_{\text{sec}}}{\text{Area}} $$

where $s$ represents the daily level of the lake (measured in $m$), $\text{inflow}$ and $\text{outflow}$ values are averaged over their corresponding day (measured in $m^3/s$), $n_\text{sec}$ is the number of seconds in a day and $\text{Area}$ represents the surface area of the lake. For the moment I found $\text{Area}=146km^2$, but a more accurate value should be needed.

In [3]:
class como_lake:
    
    def __init__(self, s0=197.37, t0=0, inflows=None):
        
        # Define initial and objective levels of water
        self.state = s0
        self.zero_state = 197.37
        
        # If there are not custom inflow values, take historic data
        if inflows == None:
            self.inflows = np.genfromtxt('data/como_data_1946_2011.txt', delimiter="  ")[1:,5]
        self.t_limit = len(self.inflows)
        self.time_id = t0     
        
        
    def step(self, outflow, inflow=None):
        
        ### Function returns:  new_state, reward, end_flag 
        
        # Take inflow value from data if not provided
        if inflow == None:
            inflow = inflows[self.time_id]
        
        # Check if:  - Outflow is positive -> if negative consider it equal to 0
        #            - There is enough water to release -> if not, clip it to the current level
        outflow = np.clip(outflow, 0, self.state*146000000/86400)
        
        # Compute new level of the lake after action and inflow
        in_out_diff = inflows[self.time_id] - outflow
        self.state += in_out_diff*86400 / 146000000
        
        # Check if there still is inflow data: if yes, update time index
        if self.time_id >= self.t_limit:
            return self.state, -np.abs(self.state-self.zero_state), True
        else:
            self.time_id += 1
            return self.state, -np.abs(self.state-self.zero_state), False

## Hyper-Policy and Policies
The policy is linear and deterministic, thus we have $a = \pi_{\theta_t}(s) = W_t s + b_t$ and $\theta_t=\{W_t,b_t\}$, but we will clip it to $0$ if it tries to release a negative quantity of water. Then, the hyper-policy will be $\nu_\rho(W_t,b_t|t) = \left( \mathcal{N}(\mu_{W,t},\Sigma_{W,t}), \mathcal{N}(\mu_{b,t},\Sigma_{b,t}) \right)$, with:
 - $\mu_{W,t} = \nu_{\rho,W}(t) = A_{W} \sin(\phi_{W} t + \psi_{W}) + B_{W}$
 - $\mu_{b,t} = \nu_{\rho,b}(t) = A_{b} \sin(\phi_{b} t + \psi_{b}) + B_{b}$
 - $\Sigma_{W,t}$ and $\Sigma_{b,t}$ given as parameters

In [4]:
class linear_policy:
    
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
    def take_action(self, state):
        return min(0, self.W*state + self.b)

In [5]:
class hyper_policy:
        
    def __init__(self, sigma_w, sigma_b, params=None):
        
        # Store sigmas of the gaussian distributions
        self.sigma_W = sigma_W
        self.sigma_b = sigma_b
        
        # If params are given, initialize hyperpolicy means with them, else take random ones
        if params == None:
            params = np.random.rand(8)
            
        self.A_w   = params[0]
        self.B_w   = params[1]
        self.phi_w = params[2]
        self.psi_w = params[3]
        
        self.A_b   = params[4]
        self.B_b   = params[5]
        self.phi_b = params[6]
        self.psi_b = params[7]

        
    def params(self):
        return [self.A_w, self.B_w, self.phi_w, self.psi_w,
                self.A_b, self.B_b, self.phi_b, self.psi_b]
    
    
    
    def W_mean(self, t):
        return self.A_w * np.sin(self.phi_w*t + self.psi_w) + self.B_w
    
    def b_mean(self, t):
        return self.A_b * np.sin(self.phi_b*t + self.psi_b) + self.B_b
    
    def W_pdf(self, W, t):
        return scipy.stats.norm.pdf(W, loc=self.W_mean(t), scale=self.sigma_W)
    
    def b_pdf(self, b, t):
        return scipy.stats.norm.pdf(b, loc=self.b_mean(t), scale=self.sigma_b)
    
    def theta_pdf(self, theta, t):
        return W_pdf(theta[0],t) * b_pdf(theta[1],t)
    
    def sample_theta(self, t):
        W = scipy.stats.norm.rvs(loc=self.W_mean(t), scale=self.sigma_W)
        b = scipy.stats.norm.rvs(loc=self.b_mean(t), scale=self.sigma_b)
        return W, b

    def sample_policy(self, t):
        W, b = self.sample_theta(t)
        return linear_policy(W,b)
    
    
    #def update_params(self, delta_params):
        
        # TO DO
        

## Metrics and Tools
Here we define three possible performance estimators of our hyperpolicy $\nu_{\rho'}$ on the last $\alpha$ samples collected by a behavioural hyperpolicy $\nu_\rho$. They are defined in the `performance_estimators.py` file and are:

 - Trajectory return estimation (IS weight based on entire trajectory):
   $$ J^1_t(\nu_\rho) = \left( \prod_{i=t-\alpha}^{t}  \frac{\nu_\rho(\theta_i\vert t+1)}{\nu_\rho(\theta_i\vert i)} \right) \left(\sum_{i=t-\alpha}^{t} \beta^{t-i} \gamma^{i-t+\alpha} R(s_i,\pi_{\theta_i}(s_i))\right) $$
 - Per-step reward estimation (IS weight based on trajectory up to time $i$):
   $$ J^2_t(\nu_\rho) =  \sum_{i=t-\alpha}^{t} \beta^{t-i} \gamma^{i-t+\alpha} R(s_i,\pi_{\theta_i}(s_i))\left( \prod_{k=t-\alpha}^{i} \frac{\nu_\rho(\theta_k\vert t+1)}{\nu_\rho(\theta_k\vert k)} \right) $$
 - Per-step reward estimation (IS weight based only on $i$-th timestep):
   $$ J^3_t(\nu_\rho) =  \sum_{i=t-\alpha}^{t} \beta^{t-i} \gamma^{i-t+\alpha} R(s_i,\pi_{\theta_i}(s_i)) \frac{\nu_\rho(\theta_i\vert t+1)}{\nu_\rho(\theta_i\vert i)} $$
A reliable estimated performance should be obtained by sampling $N$ different trajectories from the $\nu_\rho$ starting always from the state $s_{t-\alpha}$.

In [ ]:
# play a random hyperpolicy, measure its performance and check if everything works correctly

    # TO DO